# Imports

In [1]:
import json

from collections import OrderedDict

In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.dataset import TensorDataset
from torch.utils.data.dataloader import DataLoader

In [6]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# Model Imports

In [7]:
from models import Model_1
from models import Model_2
from models import Model_3
from models import Model_4

# Auxiliary Functions

# Model Example №1

## Dataset Generation

In [8]:
dataset_1 = pd.read_csv("results_1.csv")
dataset_1 = dataset_1.query("t != 1.0")

In [9]:
def make_features_1(t, x1):
    time_features = [(f"t_pow_{p}", t ** p) for p in range(1, 6)]
    state_features = [(f"x1_pow_{p}", x1 ** p) for p in range(1, 6)]
    product_features = [
        (f"{n1}_mul_{n2}", f1 * f2)
        for n1, f1 in time_features 
        for n2, f2 in state_features]
    return OrderedDict(
        time_features + 
        state_features + 
        product_features)

In [10]:
enriched_dataset_1 = pd.DataFrame([
    make_features_1(r["t"], r["x1"])
    for _, r in dataset_1.iterrows()])

enriched_dataset_1["u"] = dataset_1["u"].values

In [11]:
feature_names = [
    f 
    for f in list(enriched_dataset_1.columns) 
    if f != "u"]

target_name = ["u"]

## Dataset normalization

In [13]:
means_1 = OrderedDict()
sigmas_1 = OrderedDict()

In [14]:
for phi in feature_names:
    means_1[phi] = enriched_dataset_1[phi].mean()
    sigmas_1[phi] = enriched_dataset_1[phi].std()

    enriched_dataset_1[phi] -= means_1[phi]
    enriched_dataset_1[phi] /= sigmas_1[phi]

In [15]:
train_1, test_1 = train_test_split(enriched_dataset_1, test_size=0.1)

In [16]:
train_1_dataset = TensorDataset(
    torch.from_numpy(train_1[feature_names].values),
    torch.from_numpy(train_1[target_name].values))

train_1_loader = DataLoader(
    train_1_dataset, 
    batch_size=64, 
    shuffle=True,
    num_workers=4)

## Neural Controller

In [17]:
class Net_1(nn.Module):
    
    def __init__(self):
        super(Net_1, self).__init__()
        self.layer_1 = nn.Linear(len(feature_names), 16)
        self.layer_2 = nn.Linear(16, 1)
    
    def forward(self, x):
        x = self.layer_1(x)
        x = self.layer_2(x)
        return x

In [18]:
net_1 = Net_1()
net_1.double()

Net_1(
  (layer_1): Linear(in_features=35, out_features=16, bias=True)
  (layer_2): Linear(in_features=16, out_features=1, bias=True)
)

In [19]:
num_epochs = 250
lr = 1e-3

In [20]:
optimizer = optim.SGD(net_1.parameters(), lr=lr)
criterion = nn.L1Loss(reduction="mean")

In [21]:
for epoch_id in tqdm(range(num_epochs)):
    for batch_id, data in enumerate(train_1_loader, 0):
        inputs, targets = data
        optimizer.zero_grad()
        
        outputs = net_1(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
    if epoch_id == 0 or (epoch_id + 1) % 10 == 0:
        outputs = net_1.forward(torch.from_numpy(test_1[feature_names].values))
        test_error = criterion(outputs, torch.from_numpy(test_1[target_name].values))
        print(f"Test error on {epoch_id + 1}: {test_error}")

Test error on 1: 6.155235705473992
Test error on 10: 2.783644199302359
Test error on 20: 1.3858397715414272
Test error on 30: 0.8268540902325634
Test error on 40: 0.5438750125233998
Test error on 50: 0.36950338489998125
Test error on 60: 0.2875860983281175
Test error on 70: 0.23002924705866606
Test error on 80: 0.19021065385122068
Test error on 90: 0.17147258753073905
Test error on 100: 0.15886998786509918
Test error on 110: 0.15139706068002542
Test error on 120: 0.14768055852346335
Test error on 130: 0.1430546800882806
Test error on 140: 0.13921945898083718
Test error on 150: 0.13578410404392416
Test error on 160: 0.13484850271932988
Test error on 170: 0.13259557652721776
Test error on 180: 0.1302644481089856
Test error on 190: 0.1315266171371379
Test error on 200: 0.1282798915639394
Test error on 210: 0.12703299660915549
Test error on 220: 0.1267956614668554
Test error on 230: 0.1266349456886156
Test error on 240: 0.12494203255429323
Test error on 250: 0.12666398199478676



In [42]:
def neuro_controller_1(t, x):
    features = make_features_1(t, x[0])
    for k, v in features.items():
        features[k] -= means_1[k]
        features[k] /= sigmas_1[k]

    features = np.array(list(features.values())).astype(np.float64)
    control = net_1.forward(torch.from_numpy(features)).item()
    return control

In [67]:
initial_state = np.array([37.7])

In [68]:
optimal_model = Model_1()
grid = np.linspace(optimal_model.t0, optimal_model.t1, num=101)

In [69]:
optimal_sol = optimal_model.calc_solution(initial_state, grid)
optimal_control = optimal_model.get_controls(optimal_sol)

In [70]:
optimal_I = optimal_model.I(optimal_sol) + optimal_model.I_term(optimal_sol)

In [71]:
suboptimal_model = Model_1(neuro_controller_1)

In [72]:
suboptimal_sol = suboptimal_model.calc_solution(initial_state, grid)
suboptimal_control = suboptimal_model.get_controls(suboptimal_sol)

In [73]:
suboptimal_I = suboptimal_model.I(suboptimal_sol) + suboptimal_model.I_term(suboptimal_sol)

In [74]:
print(f"Optimal I: {np.round(optimal_I, 5)}")
print(f"Suboptimal I: {np.round(suboptimal_I, 5)}")

Optimal I: 355.3225
Suboptimal I: 356.8468


In [84]:
fig_optimal_state = go.Scatter(
    x=optimal_sol.t,
    y=optimal_sol.y[0, :], 
    name="x1 - optimal", 
    yaxis="y1",
    line=dict(color="black"))

fig_suboptimal_state = go.Scatter(
    x=suboptimal_sol.t,
    y=suboptimal_sol.y[0, :], 
    name="x1 - suboptimal", 
    yaxis="y1",
    line=dict(color="black", dash="dash"))

difference = suboptimal_sol.y[0, :] - optimal_sol.y[0, :]
max_dif = np.abs(difference).max()
fig_difference_state = go.Scatter(
    x=optimal_sol.t,
    y=difference,
    name="difference", 
    yaxis="y2",
    line=dict(color="black", dash="dot"))

In [85]:
iplot(
    go.Figure(
        data=[
            fig_optimal_state,
            fig_suboptimal_state,
            fig_difference_state
        ],
        layout=go.Layout(
            legend=dict(orientation="h", x=0.1, y=1.1),
            xaxis=dict(
                range=(optimal_model.t0, optimal_model.t1)
            ),
            yaxis1=dict(
                range=(-initial_state[0], initial_state[0])
            ),
            yaxis2=dict(
                range=(-max_dif, max_dif),
                overlaying="y1",
                side="right"
            )
        )
    )
)

In [86]:
fig_optimal_control = go.Scatter(
    x=optimal_sol.t,
    y=optimal_control, 
    name="u - optimal", 
    yaxis="y1",
    line=dict(color="black"))

fig_suboptimal_control = go.Scatter(
    x=suboptimal_sol.t,
    y=suboptimal_control, 
    name="u - suboptimal", 
    yaxis="y1",
    line=dict(color="black", dash="dash"))

difference = np.array(suboptimal_control) - np.array(optimal_control)
max_dif = np.abs(difference).max()
fig_difference_control = go.Scatter(
    x=optimal_sol.t,
    y=difference,
    name="difference", 
    yaxis="y2",
    line=dict(color="black", dash="dot"))

In [87]:
iplot(
    go.Figure(
        data=[
            fig_optimal_control,
            fig_suboptimal_control,
            fig_difference_control
        ],
        layout=go.Layout(
            legend=dict(orientation="h", x=0.1, y=1.1),
            xaxis=dict(
                range=(optimal_model.t0, optimal_model.t1)
            ),
            yaxis1=dict(
                range=(-initial_state[0], initial_state[0])
            ),
            yaxis2=dict(
                range=(-max_dif, max_dif),
                overlaying="y1",
                side="right"
            )
        )
    )
)